In [1]:
import glob
path =r'H:\REDDIT\DATA\reddit-top-2.5-million\data' # use your path
allFiles = glob.glob(path + "/*.csv")

In [2]:
# select 500 random files
import random
trainFiles = random.sample(range(len(allFiles)), 500)

In [3]:
import pandas
frame = pandas.DataFrame()
list_ = []
for file_ in trainFiles:
    df = pandas.read_csv(allFiles[file_],index_col=None, header=0)
    list_.append(df)
frame = pandas.concat(list_)

In [42]:
len(trainFiles)

500

In [4]:
# train data set
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
count_vect = CountVectorizer(stop_words='english')
X_train_counts = count_vect.fit_transform(frame['title'])
tf_transformer = TfidfTransformer().fit(X_train_counts)
X_train_counts_tf = tf_transformer.transform(X_train_counts)
X_train_out = frame['score']

In [6]:
from sklearn.tree import DecisionTreeRegressor
reg = DecisionTreeRegressor(max_depth = 25)
reg.fit(X_train_counts_tf, X_train_out)

DecisionTreeRegressor(criterion='mse', max_depth=25, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')

In [15]:
from sklearn.metrics import mean_squared_error
X_train_predict = reg.predict(X_train_counts_tf)

In [16]:
mean_squared_error(X_train_out, X_train_predict)

114869.71644989881

In [10]:
# check mean squared error on 500 different subreddits than used for training
frame_test = pandas.DataFrame()
list_ = []
for file_ in range(len(allFiles)):
    if file_ not in trainFiles:
        df = pandas.read_csv(allFiles[file_],index_col=None, header=0)
        list_.append(df)
    if len(list_) == 500:
        break
frame_test = pandas.concat(list_)

In [19]:
X_test_out = frame_test['score']
X_test_counts = count_vect.transform(frame_test['title'])
X_test_counts_tf = tf_transformer.transform(X_test_counts)
X_test_predict = reg.predict(X_test_counts_tf)

In [20]:
mean_squared_error(X_test_out, X_test_predict)

168460.98168692968

In [21]:
# utility function for predicting score of lineNum row data in fileName
def predictScore(fileName, lineNum):
    dfp = pandas.read_csv(fileName, index_col=None, header=0)
    X_counts = count_vect.transform(dfp['title'])
    X_counts_tf = tf_transformer.transform(X_counts)
    X_predict = reg.predict(X_counts_tf)
    return X_predict[lineNum]

In [41]:
predictScore(path+"/coding.csv", 10)

146.44141265290827

In [44]:
"""
For experimenting with GridSearch
from sklearn.grid_search import GridSearchCV
from sklearn import metrics
from sklearn.metrics import make_scorer
from sklearn.tree import DecisionTreeRegressor

regressor = DecisionTreeRegressor()
parameters = {'max_depth':(10,20,25)}
scoring_function = make_scorer(mean_squared_error, greater_is_better = False)
reg = GridSearchCV(regressor, parameters, scoring_function)
reg.fit(X_train_counts_tf, X_train_out)
"""

"\nfrom sklearn.grid_search import GridSearchCV\nfrom sklearn import metrics\nfrom sklearn.metrics import make_scorer\nfrom sklearn.tree import DecisionTreeRegressor\n\nregressor = DecisionTreeRegressor()\nparameters = {'max_depth':(10,20,25)}\nscoring_function = make_scorer(mean_squared_error, greater_is_better = False)\nreg = GridSearchCV(regressor, parameters, scoring_function)\nreg.fit(X_train_counts_tf, X_train_out)\n"

In [ ]:
one_hot = pandas.get_dummies(pd['domain'], sparse=True)
hot_header_list = one_hot.columns.values.tolist()
in1 = pd[['over_18']+ hot_header_list]
out1 = pd['score']

In [ ]:
reg = DecisionTreeRegressor(max_depth = 10)

In [ ]:
reg.fit(in1_train, out1_train)

In [ ]:
reg.predict(in1_test)

In [ ]:
out1_test

In [ ]:
in1_train.shape

In [ ]:
out1_train = pd['score'].values[1:501:1]

In [ ]:
out1_train.shape

In [ ]:
#from sklearn.linear_model import LinearRegression

In [ ]:
#reg = LinearRegression()

In [ ]:
in1_test = in1.values[500::1]

In [ ]:
in1_test.shape

In [ ]:
out1_test = pd['score'].values[500:1000:1]

In [ ]:
out1_test.shape

In [ ]:
out1_test

In [ ]:
pd['title'][10]

In [ ]:
len(pd.index)

In [ ]:
pd['title'][10].split()

In [ ]:
from pandas import *
files = !ls *.csv # IPython magic

In [ ]:
d = concat([read_csv(f, index_col=0, header=None, axis=1) for f in files], keys=files)

In [ ]:
pwd

In [ ]:
import os
import glob
import pandas

In [ ]:
my_dir = "H:\\REDDIT\\try"
filelist = []
os.chdir( my_dir )
for files in glob.glob( "*.csv" ) :
    filelist.append(files)

In [ ]:
filelist

In [ ]:
frame.keys()

In [ ]:
frame['subreddit_id']